In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/shashankkumar/Documents/GitHub/MacroEcon/data/step2/process') # to import from address.py in houehold.py
from genericpath import exists
from os import makedirs

In [2]:
from process.base_pop import base_pop_wrapper
from process.household import household_wrapper

# Step 1: Generate Synthetic Agent Population

In [3]:
OUTPUT_DIR = '/Users/shashankkumar/Documents/GitHub/MacroEcon'
if not exists(OUTPUT_DIR):
    makedirs(OUTPUT_DIR)

AGENTS_DF_PATH = '../step1/all_nta_agents.npy' # Dictionary created from excel files
household_data_path = "/Users/shashankkumar/Documents/GitHub/MacroEcon/housing_v2.pkl"  # Household data from excel file
base_pop_out_file = '/base_population1.pkl' # Output file for base population

geo_address_data = None # Geo address data from excel file if exists (long, lat). The location (for every entity) for each geographic area (NTA)
use_parallel = True # Whether to use Ray. Household generation doesn't work with Ray
n_cpu = 8 # Number of CPU cores to use for parallel processing

# For Household generation Only
adult_list = ['20t29','30t39', '40t49', '50t64', '65A']
children_list = ['U19']

In [4]:
# Input for base population
agents_data = np.load(AGENTS_DF_PATH, allow_pickle=True).item()

# For Household generation
household_data = pd.read_pickle(household_data_path) # Household data from excel file

# Area Filter
area_selector = ['BK0101'] # List of areas to use, population for only these areas are generated. If None, all areas are used

#### Base Population

In [5]:
# Base address is currently not used
base_population,base_address = base_pop_wrapper(input_data=agents_data['valid_ntas'],input_mapping=agents_data['mapping'],use_parallel=True,n_cpu=8,area_selector=area_selector)
base_population.to_pickle(OUTPUT_DIR + base_pop_out_file)

2024-03-14 01:25:17,838	INFO worker.py:1724 -- Started a local Ray instance.


#### Add Household

In [6]:
use_parallel = False

base_population,base_address = household_wrapper(
    household_data,
    base_population,
    base_address = None,
    adult_list=adult_list,
    children_list=children_list,
    geo_address_data=None,
    use_parallel=use_parallel,
    n_cpu=n_cpu,
)

base_population.to_pickle(OUTPUT_DIR + base_pop_out_file)

/Users/shashankkumar/Documents/GitHub/MacroEcon/data/step2/process/household.py:375: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_household_data["adult_num"] = (
/Users/shashankkumar/Documents/GitHub/MacroEcon/data/step2/process/household.py:379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_household_data["adult_num"] = proc_household_data["adult_num"].clip(lower=0)
/Users/shashankkumar/Documents/GitHub/MacroEcon/data/step2/process/household.py:195: SettingWithCopyWarning: 
A value is trying to 

In [7]:
base_population.head()

,area,age,gender,ethnicity,education,employment_insurance,household
0,BK0101,U19,Male,white,college_degree,employed_insured,4941.0
1,BK0101,U19,Male,white,graduate_degree,employed_insured,8957.0
2,BK0101,U19,Male,white,graduate_degree,employed_insured,5058.0
3,BK0101,U19,Female,hispanic,graduate_degree,unemployed_uninsured,6824.0
4,BK0101,U19,Female,other,high_school,employed_insured,971.0
